## This notebook consists of scripts 

    • To convert PASCAL VOC file from .xml format to .csv
    • Script to split csv dataset to 'test' and 'train'
    • Script to train using Pytorch 
    • Test the data using 'test' dataset
    

#### Import all the libraries

In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import re

from PIL import Image

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from albumentations.pytorch import transforms

import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

from matplotlib import pyplot as plt

#### Convert the xml to csv file

In [ ]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


# def main():
train_test_path = r'/Users/praveen/Desktop/notebook_/perfume/example_small_data'
save = r'/Users/praveen/Desktop/notebook_/perfume'  # for directory in ['train', 'test']:

xml_df = xml_to_csv(train_test_path)
xml_df.to_csv(save + '/' + 'notebook_annotations.csv')


#### Load the datatset path

In [27]:
DIR_INPUT = '/Users/praveen/Desktop/notebook_/perfume/example_small_data'
DIR_TRAIN = f'{DIR_INPUT}'
DIR_TEST = f'{DIR_INPUT}'

train_dir = '/Users/praveen/Desktop/notebook_/perfume/example_small_data'

In [28]:
df = pd.read_csv(f'{DIR_INPUT}/notebook_annotations.csv')

#### Split the data into valid and train

In [ ]:
msk = np.random.rand(len(train_df)) < 0.8
df = df.rename(columns={'filename': 'image_id'})

train_df = df[msk]
valid_df = df[~msk]

#### Class to read and return data in specific format for training

In [7]:
class ProjectDataset(Dataset):

    def __init__(self, dataframe, image_dir, transforms=None,train=True):
        super().__init__()

        self.image_ids = dataframe['image_id'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms
        self.train=train

    def __len__(self) -> int:
        return self.image_ids.shape[0]

    def __getitem__(self, index: int):

        image_id = self.image_ids[index]
        image = cv2.imread(f'{self.image_dir}/{image_id}', cv2.IMREAD_COLOR)
        print(image.shape)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

            
        if(self.train==False):  # For test data
            return image, image_id
        

        
        #Else for train and validation data
        records = self.df[self.df['image_id'] == image_id]   
        
        boxes = records[['xmin', 'ymin', 'xmax', 'ymax']].values
       
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)

        # there is only one class
        labels = torch.ones((records.shape[0],), dtype=torch.int64)
        
        # suppose all instances are not crowd
        iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])
        target['area'] = area
        target['iscrowd'] = iscrowd
        
        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']

        return image, target,image_id  

#### Apply Image Augmentations

In [8]:
# Albumentations
def get_train_transform():
    return A.Compose([
        A.Flip(0.5),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

#### Print loss for current iteration

In [9]:
class Averager:      ##Return the average loss 
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

#### Create iteratable dataset

In [10]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_dataset = ProjectDataset(train_df, train_dir, get_train_transform(),True)
valid_dataset = ProjectDataset(valid_df, train_dir, get_train_transform(),True)


train_data_loader = DataLoader(
    train_dataset,
    batch_size=4,    # change the batch size later
    shuffle=False,
    num_workers=0,
    collate_fn=collate_fn
)

valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=2,    # change the batch size later
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

#### Simulate on GPU, if available

In [11]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

#### Load the data into variables

In [ ]:
images, targets, image_ids = next(iter(train_data_loader))

#### Plot and check the data with bbox

In [ ]:
images = list(image.to(device) for image in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

boxes = targets[4]['boxes'].cpu().numpy().astype(np.int32)
sample = images[4].permute(1,2,0).cpu().numpy()

fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(sample,
                  (box[0], box[1]),
                  (box[2], box[3]),
                  (220, 0, 0), 3)
    
ax.set_axis_off()
ax.imshow(sample)

#### Load the weights from architeture

In [ ]:
# load a model; pre-trained on COCO
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

#### Set the number of classes and Load the features 

In [15]:
num_classes = 4  # 1 class (wheat) + background

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

#### Set the hyperparams

In [19]:
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
lr_scheduler = None



#### Itearate and train the model

In [ ]:
num_epochs = 1
itr = 1
loss_hist = Averager()


for epoch in range(num_epochs):
    loss_hist.reset()
    
    for images, targets, image_ids in train_data_loader:
        
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()

        loss_hist.send(loss_value)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if itr % 50 == 0:
            print(f"Iteration #{itr} loss: {loss_value}")

        itr += 1
    
    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()

    print(f"Epoch #{epoch} loss: {loss_hist.value}")   

#### Load the valid or test dataset to evaluate

In [ ]:
images, targets, image_ids = next(iter(valid_data_loader))
images = list(img.to(device) for img in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
boxes = targets[1]['boxes'].cpu().numpy().astype(np.int32)
sample = images[1].permute(1,2,0).cpu().numpy()

model.eval()
cpu_device = torch.device("cpu")

outputs = model(images)
outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]

#### Plot the results

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(sample,
                  (box[0], box[1]),
                  (box[2], box[3]),
                  (220, 0, 0), 3)
    
ax.set_axis_off()
ax.imshow(sample)

#### Save the trained weights

In [ ]:
torch.save(model.state_dict(), 'fasterrcnn_resnet50_fpn.pth')